## Homework

In this homework, we'll deploy the bees vs wasps model we trained in the 
[previous homework](../08-deep-learning/homework.md).

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

In [1]:
from pathlib import Path
from urllib.parse import urlparse
import os

url_model = "https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5"
path_model = os.path.join('/workspace', Path(urlparse(url_model).path).name)

In [11]:
%%bash -s "$url_model" "$path_model"
cd "/workspace"
url_model=$1
path_model=$2


if [ ! -f "$path_model" ]; then
    # use curl for downloading
    echo "File not found! Downloading it"
    curl -LJO -o $path_model $url_model
else
    echo "File already exists. Skipping download."
fi

File not found! Downloading it


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 85.5M  100 85.5M    0     0  17.8M      0  0:00:04  0:00:04 --:--:-- 21.8M


---

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb


In [12]:
path_model

'/workspace/bees-wasps.h5'

In [13]:
import tensorflow as tf

model = tf.keras.models.load_model(path_model)
model.summary()

2023-11-23 17:50:12.141456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-23 17:50:12.164371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-23 17:50:12.164415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-23 17:50:12.167058: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-23 17:50:12.167098: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [14]:
path_tflite = '/workspace/model.tflite'
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) 
tflite_model = converter.convert()

# Save the model.
with open(path_tflite, 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp4sl6dodm/assets


INFO:tensorflow:Assets written to: /tmp/tmp4sl6dodm/assets
2023-11-23 17:50:19.274031: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-23 17:50:19.274093: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-23 17:50:19.275377: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4sl6dodm
2023-11-23 17:50:19.276240: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-23 17:50:19.276252: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4sl6dodm
2023-11-23 17:50:19.279654: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-23 17:50:19.280598: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-23 17:50:19.372981: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [15]:
# retrieve file size 
import os
size_tflite = os.path.getsize(path_tflite)
size_model = os.path.getsize(path_model)
size_tflite/size_model

0.4998804285462295

TF-Lite model has only have the size of the original keras model.

In [16]:
round(size_tflite / 1024**2)

43

In [17]:
!ls /workspace/model.tflite -lah

-rw-r--r-- 1 1000 1000 43M Nov 23 17:50 /workspace/model.tflite


### Answer 1

`43`

---

## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* 13
* 24


In [18]:
# retrieve index for input and output of the tflite_model

interpreter = tf.lite.Interpreter(model_path=path_tflite)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_index = input_details[0]['index']
output_index = output_details[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [19]:
output_index

13

### Answer 2

`13`

---

## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image: 

https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

Based on the previous homework, what should be the target size for the image?


In [20]:
url_img = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
path_img = Path(urlparse(url_img).path).name

In [21]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [22]:
input_details[0]['shape']

array([  1, 150, 150,   3], dtype=int32)

In [23]:
img = download_image(url_img)
img = prepare_image(img, target_size=(150,150))

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3450980
* 0.5450980
* 0.7450980
* 0.9450980

In [24]:
type(img)

PIL.Image.Image

In [25]:
import numpy as np

if not isinstance(img, np.ndarray):
    # convert PIL image to numpy array
    img = np.array(img)

img = img.astype(float) / 255.0

In [26]:
print(f"{img[0,0,0]:.7f}")

0.9450980


### Answer 3

`0.9450980`

---

## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.258
* 0.458
* 0.658
* 0.858


In [27]:
# Prepare input data
input_data = tf.convert_to_tensor(np.expand_dims(img, axis=0), dtype=tf.float32)

# Set the input tensor
interpreter.set_tensor(input_index, input_data)

# Invoke the model
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_index)

# Print the output shape and value
print(output_data.shape)
print(output_data)

(1, 1)
[[0.6592137]]


In [28]:
round(output_data[0][0],3)

0.659

### Answer 4

`0.658`

---

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10
COPY bees-wasps-v2.tflite .
```

And pushed it to [`agrigorev/zoomcamp-bees-wasps:v2`](https://hub.docker.com/r/agrigorev/zoomcamp-bees-wasps/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The version of Python is 3.10, so you need to use the right wheel for 
  TF-Lite. For Tensorflow 2.14.0, it's https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


---

## Question 5

Download the base image `agrigorev/zoomcamp-bees-wasps:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 162 Mb
* 362 Mb
* 662 Mb
* 962 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.


```bash
# command line from local system not the previous used docker container
docker_image=agrigorev/zoomcamp-bees-wasps:v2
docker pull $docker_image 
docker images --format "{{.Size}}" $docker_image
```


### Answer 5

`662MB`

---

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `bees-wasps-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).

Now run the container locally.

Score this image: https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

What's the output from the model?

* 0.2453
* 0.4453
* 0.6453
* 0.8453


```bash
# run on the console, not inside this docker container
docker compose build aws-lambda
docker compose run --rm --service-ports aws-lambda

echo '''{
    "url": "https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg"
}
''' > data.json

# curl version 7.8.1
cat data.json | curl \
    -X POST -H "Content-Type: application/json" \
    -d @- http://localhost:8080/2015-03-31/functions/function/invocations

# curl version >= 7.8.2
cat data.json | curl --json @- http://localhost:8080/2015-03-31/functions/function/invocations

```


```
````

### Answer 6

```json
{"prediction": 0.35206058621406555}
```

---

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.


## Publishing to Docker hub

This is just for reference, this is how we published our image to Docker hub:

```bash
docker build -t zoomcamp-bees-wasps -f homework.dockerfile .
docker tag zoomcamp-bees-wasps:latest agrigorev/zoomcamp-bees-wasps:v2
docker push agrigorev/zoomcamp-bees-wasps:v2
```


## Submit the results

- Submit your results here: https://forms.gle/CrgtJVVs64DTGbCp9
- If your answer doesn't match options exactly, select the closest one
- You can submit your solution multiple times. In this case, only the last submission will be used


## Deadline

The deadline for submitting is November 27 (Monday), 23:00 CET. After that the form will be closed.
